# Join Statements - Lab done by `Eugene Maina`

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
# Your code here
names = '''SELECT firstName || ' ' || lastName AS 'Full Name' 
            FROM employees
            JOIN offices
            ON employees.officeCode = offices.officeCode
            WHERE city = "Boston";'''
pd.read_sql(names, conn)

,Full Name
0,Julie Firrelli
1,Steve Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:
# Your code here
zero_employees = '''SELECT officeCode, city, COUNT(employeeNumber) AS 'Number of Employees'
                    FROM offices
                    LEFT JOIN employees
                    USING(officeCode)
                    GROUP BY officeCode
                    HAVING COUNT(employeeNumber) = 0;'''
pd.read_sql(zero_employees, conn)

,officeCode,city,Number of Employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [12]:
"""
Which 10 employees have the most customers?
"""

# Your code here
customers_per_employee = '''SELECT employeeNumber, firstName || ' ' || lastName AS 'Full Name', COUNT(customerNumber) AS 'Number of Customers'
                            FROM employees
                            JOIN customers
                            ON employees.employeeNumber = customers.salesRepEmployeeNumber
                            GROUP BY employeeNumber
                            ORDER BY COUNT(customerNumber) DESC
                            LIMIT 10;'''
pd.read_sql(customers_per_employee, conn, index_col='employeeNumber')

,Full Name,Number of Customers
employeeNumber,,
1401,Pamela Castillo,10
1504,Barry Jones,9
1501,Larry Bott,8
1323,George Vanauf,8
1370,Gerard Hernandez,7
1286,Foon Yue Tseng,7
1702,Martin Gerard,6
1337,Loui Bondur,6
1216,Steve Patterson,6


In [11]:
"""
Which 10 customers have the highest payments?
"""

# Your code here
highest_payments = '''SELECT customerNumber, customerName, SUM(amount) AS 'Total Payments'
                    FROM customers
                    JOIN payments
                    USING(customerNumber)
                    GROUP BY customerNumber
                    ORDER BY SUM(amount) DESC
                    LIMIT 10;'''
pd.read_sql(highest_payments, conn, index_col='customerNumber')



,customerName,Total Payments
customerNumber,,
141,Euro+ Shopping Channel,715738.98
124,Mini Gifts Distributors Ltd.,584188.24
114,"Australian Collectors, Co.",180585.07
151,Muscle Machine Inc,177913.95
148,"Dragon Souveniers, Ltd.",156251.03
323,"Down Under Souveniers, Inc",154622.08
187,"AV Stores, Co.",148410.09
276,"Anna's Decorations, Ltd",137034.22
321,Corporate Gift Ideas Co.,132340.78


In [ ]:
"""
Question: What are the top 10 products ordered?
"""

# Your code here
most_orders = '''SELECT productCode, productName, COUNT(orderNumber) AS 'Number of Orders'
                FROM products
                JOIN orderdetails
                USING(productCode)
                GROUP BY productCode
                ORDER BY COUNT(orderNumber) DESC
                LIMIT 10;'''
pd.read_sql(most_orders, conn, index_col='productCode')

,productName,Number of Orders
productCode,,
S18_3232,1992 Ferrari 360 Spider red,53
S72_1253,Boeing X-32A JSF,28
S700_4002,American Airlines: MD-11S,28
S700_3167,F/A 18 Hornet 1/72,28
S700_2834,ATA: B757-300,28
S700_2824,1982 Camaro Z28,28
S700_2466,America West Airlines B757-200,28
S700_2047,HMS Bounty,28
S700_1691,American Airlines: B767-300,28


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [19]:
# Your code here
products_by_employee = '''SELECT employeeNumber, firstName || ' ' || lastName AS 'Full Name', productName, COUNT(productCode) AS 'Number of Products Sold'
                        FROM employees
                        JOIN customers
                        ON employees.employeeNumber = customers.salesRepEmployeeNumber
                        JOIN orders
                        USING(customerNumber)
                        JOIN orderdetails
                        USING(orderNumber)
                        JOIN products
                        USING(productCode)
                        GROUP BY employeeNumber, productCode
                        ORDER BY COUNT(productCode) DESC;'''
pd.read_sql(products_by_employee, conn, index_col='employeeNumber')

,Full Name,productName,Number of Products Sold
employeeNumber,,,
1370,Gerard Hernandez,1992 Ferrari 360 Spider red,11
1165,Leslie Jennings,1917 Maxwell Touring Car,8
1323,George Vanauf,1997 BMW F650 ST,7
1370,Gerard Hernandez,1970 Dodge Coronet,7
1370,Gerard Hernandez,1958 Chevy Corvette Limited Edition,7
...,...,...,...
1702,Martin Gerard,The Mayflower,1
1702,Martin Gerard,HMS Bounty,1
1702,Martin Gerard,America West Airlines B757-200,1


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [21]:
# Your code here
sold_products_per_employee = '''SELECT employeeNumber, firstName || ' ' || lastName AS 'Full Name', COUNT(productCode) AS 'Number of Products Sold'
                            FROM employees
                            JOIN customers
                            ON employees.employeeNumber = customers.salesRepEmployeeNumber
                            JOIN orders
                            USING(customerNumber)
                            JOIN orderdetails
                            USING(orderNumber)
                            GROUP BY employeeNumber
                            ORDER BY lastName;'''
pd.read_sql(sold_products_per_employee, conn, index_col='employeeNumber')

,Full Name,Number of Products Sold
employeeNumber,,
1337,Loui Bondur,177
1501,Larry Bott,236
1401,Pamela Castillo,272
1188,Julie Firrelli,124
1611,Andy Fixter,185
1702,Martin Gerard,114
1370,Gerard Hernandez,396
1165,Leslie Jennings,331
1504,Barry Jones,220


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [22]:
# Your code here
over_200_sales = '''SELECT employeeNumber, firstName || ' ' || lastName AS 'Full Name', COUNT(productCode) AS 'Number of Products Sold'
                    FROM employees
                    JOIN customers
                    ON employees.employeeNumber = customers.salesRepEmployeeNumber
                    JOIN orders
                    USING(customerNumber)
                    JOIN orderdetails
                    USING(orderNumber)
                    GROUP BY employeeNumber
                    HAVING COUNT(productCode) > 200;'''
pd.read_sql(over_200_sales, conn, index_col='employeeNumber')

,Full Name,Number of Products Sold
employeeNumber,,
1165,Leslie Jennings,331
1323,George Vanauf,211
1370,Gerard Hernandez,396
1401,Pamela Castillo,272
1501,Larry Bott,236
1504,Barry Jones,220


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!